In [4]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
import yaml

# ------------------------------------------------------------------
# 1 - Carregar configurações e dados
# ------------------------------------------------------------------
with open("../config.yaml", "r") as file:
    config = yaml.safe_load(file)

train_path = config['output_data']['train_file']
test_path = config['output_data']['test_file']

print(f"Carregando dados de:\n  {train_path}\n  {test_path}")

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

# ------------------------------------------------------------------
# 2 - Preparar X (Features) e y (Target)
# ------------------------------------------------------------------
# IMPORTANTE: Não vamos filtrar apenas os IDs. Vamos usar TODAS as colunas 
# geradas pelo One-Hot Encoding no notebook anterior.

target_col = 'pairing_quality'

# Remover colunas que não são features numéricas (strings ou labels)
cols_to_drop = ['pairing_quality', 'quality_label', 'description']

# Garantir que apenas colunas existentes sejam removidas
features_train = train_df.drop(columns=[c for c in cols_to_drop if c in train_df.columns])
target_train = train_df[target_col]

features_test = test_df.drop(columns=[c for c in cols_to_drop if c in test_df.columns])
target_test = test_df[target_col]

print(f"Treinando com {features_train.shape[1]} features (incluindo One-Hot Encoding)...")

# ------------------------------------------------------------------
# 3 - Treinar Modelo (Gradient Boosting - Alternativa nativa ao XGBoost)
# ------------------------------------------------------------------
# Gradient Boosting é excelente para dados tabulares e deve baixar o RMSE.
model = GradientBoostingRegressor(
    n_estimators=300,      # Número de árvores (aumente se necessário)
    learning_rate=0.1,     # Taxa de aprendizado
    max_depth=5,           # Profundidade das árvores
    random_state=42,
    loss='squared_error'
)

print("Iniciando treinamento do modelo...")
model.fit(features_train, target_train)

# ------------------------------------------------------------------
# 4 - Avaliação
# ------------------------------------------------------------------
predictions = model.predict(features_test)

# Garantir que as predições fiquem dentro da escala 1-5 (opcional, mas recomendado)
predictions = np.clip(predictions, 1, 5)

rmse = sqrt(mean_squared_error(target_test, predictions))
mae = mean_absolute_error(target_test, predictions)

print("\n📊 Avaliação do Modelo (Gradient Boosting):")
print(f"  RMSE: {rmse:.4f}")
print(f"  MAE : {mae:.4f}")

if rmse < 1.0:
    print("\n✅ Sucesso! O RMSE foi reduzido para menos de 1.0.")
else:
    print("\n⚠️ O RMSE ainda está alto. Tente aumentar 'n_estimators' ou 'max_depth'.")

Carregando dados de:
  ../data/clean/train_file.csv
  ../data/clean/test_file.csv
Treinando com 102 features (incluindo One-Hot Encoding)...
Iniciando treinamento do modelo...

📊 Avaliação do Modelo (Gradient Boosting):
  RMSE: 1.4685
  MAE : 1.2356

⚠️ O RMSE ainda está alto. Tente aumentar 'n_estimators' ou 'max_depth'.
